In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from hyperas import optim
from keras.utils import np_utils
from hyperopt import Trials, STATUS_OK, tpe
from tensorflow.keras.models import Sequential
from hyperas.distributions import choice, uniform
from tensorflow.keras.layers import Dense, Dropout, Activation, MaxPooling2D
from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, Dropout, GlobalMaxPooling2D, BatchNormalization

Using TensorFlow backend.


In [2]:
def data():
    cifar10 = tf.keras.datasets.cifar10
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0
    y_train, y_test = y_train.flatten(), y_test.flatten()
    return x_train, y_train, x_test, y_test

In [3]:
def create_model(x_train, y_train, x_test, y_test):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation = 'relu', padding = 'same', input_shape = (32, 32, 3)))
    model.add(BatchNormalization())
    model.add(Conv2D(32, (3, 3), activation = 'relu', padding = 'same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout({{uniform(0, 1)}}))
    model.add(Conv2D(64, (3, 3), activation = 'relu', padding = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, (3, 3), activation = 'relu', padding = 'same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout({{uniform(0, 1)}}))
    model.add(Conv2D(128, (3, 3), activation = 'relu', padding = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, (3, 3), activation = 'relu', padding = 'same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout({{uniform(0, 1)}}))

    if {{choice(['three', 'four'])}} == 'four':
        model.add(Conv2D(128, (3, 3), activation = 'relu', padding = 'same'))
        model.add(BatchNormalization())
        model.add(MaxPooling2D((2, 2)))
        model.add(Dropout({{uniform(0, 1)}}))

    model.add({{choice([GlobalMaxPooling2D(), Flatten()])}})

    model.add({{choice([Dropout(0.5), Activation('linear')])}})
    model.add(Dense(1024, activation = 'relu'))
    model.add(Dropout({{uniform(0, 1)}}))
    model.add(Dense(10, activation = 'softmax'))

    model.compile(optimizer = {{choice(['rmsprop', 'adam', 'sgd'])}},
              loss = 'sparse_categorical_crossentropy',
              metrics = ['accuracy'])

    r = model.fit(x_train, y_train, validation_split = 0.1, epochs = 50)

    validation_acc = np.amax(r.history['val_accuracy'])
    print('Best validation acc of epoch:', validation_acc)
    return {'loss': -validation_acc, 'status': STATUS_OK, 'model': model} 

In [4]:
best_run, best_model = optim.minimize(model = create_model,
                                          data = data,
                                          algo = tpe.suggest,
                                          max_evals = 5,
                                          trials = Trials(),
                                          notebook_name = "hyperas_example")
    
X_train, Y_train, X_test, Y_test = data()
print("Evalutation of best performing model:")
print(best_model.evaluate(X_test, Y_test))
print("Best performing model chosen hyper-parameters:")
print(best_run)

>>> Imports:
#coding=utf-8

try:
    import numpy as np
except:
    pass

try:
    import tensorflow as tf
except:
    pass

try:
    import matplotlib.pyplot as plt
except:
    pass

try:
    from hyperas import optim
except:
    pass

try:
    from keras.utils import np_utils
except:
    pass

try:
    from hyperopt import Trials, STATUS_OK, tpe
except:
    pass

try:
    from tensorflow.keras.models import Sequential
except:
    pass

try:
    from hyperas.distributions import choice, uniform
except:
    pass

try:
    from tensorflow.keras.layers import Dense, Dropout, Activation
except:
    pass

try:
    from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, Dropout, GlobalMaxPooling2D, BatchNormalization
except:
    pass

>>> Hyperas search space:

def get_space():
    return {
        'Dropout': hp.uniform('Dropout', 0, 1),
        'Dropout_1': hp.uniform('Dropout_1', 0, 1),
        'Dropout_2': hp.uniform('Dropout_2', 0, 1),
        'Dropout_3': hp.choice('Dropout_

job exception: name 'MaxPooling2D' is not defined



  0%|          | 0/5 [00:00<?, ?trial/s, best loss=?]


NameError: name 'MaxPooling2D' is not defined